In [64]:
import numpy as np
import pandas as pd
import sqlite3
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [21]:
# # Esatblishing connection with db
# conn = sqlite3.connect("database.sqlite")
# cur = conn.cursor()

# # Checking the values in 'Player Attribute' table
# cur.execute("""SELECT * FROM Player_Attributes LIMIT 3""")
# for row in cur:
#     print(row)

# # Checking the names of all tables in the db    
# cur.execute("""SELECT name FROM sqlite_schema WHERE type='table' ORDER BY name""")
# for row in cur:
#     print(row)

# # Saving all column names on 'Player Attribut' table 
# table_col_names = []
# cur.execute("""PRAGMA table_info(Player_Attributes)""")
# for row in cur:
#     table_col_names.append(row[1])

# # Closing connection
# cur.close()
# conn.close()

In [23]:
print(len(table_col_names))
# table_col_names

42


In [24]:
conn = sqlite3.connect('database.sqlite')
df_raw = pd.read_sql_query("SELECT * FROM Player_Attributes", conn)
conn.close()

In [26]:
df = df_raw.copy()

In [27]:
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [29]:
df.shape

(183978, 42)

In [82]:
# Build model using sk
# Check important features using stats model?
# Multicolinearity?
# Check model assumptions from textbook
# Recursive feature elimination

In [31]:
df_50 = df.sample(n = 50000)

In [36]:
# Manually selecting features (34)
features = ['potential', 'crossing', 'finishing', 'heading_accuracy',
           'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
           'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
           'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
           'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
           'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
           'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
           'gk_reflexes']
target = ['overall_rating']

In [38]:
# df_50.info()

In [40]:
df_50 = df_50.dropna()
df_50.shape

(49012, 42)

In [41]:
X = df_50[features]
y = df_50[target]

In [45]:
X.head()

,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
92296,69.0,64.0,62.0,43.0,62.0,49.0,68.0,69.0,68.0,60.0,...,63.0,61.0,16.0,27.0,29.0,13.0,8.0,6.0,7.0,6.0
50343,83.0,50.0,44.0,57.0,73.0,50.0,72.0,49.0,57.0,66.0,...,67.0,59.0,54.0,59.0,47.0,13.0,7.0,7.0,12.0,10.0
107417,64.0,58.0,45.0,52.0,55.0,52.0,63.0,47.0,56.0,54.0,...,54.0,41.0,36.0,42.0,28.0,10.0,6.0,11.0,12.0,9.0
118031,72.0,47.0,75.0,77.0,66.0,70.0,62.0,49.0,39.0,41.0,...,68.0,68.0,22.0,29.0,17.0,3.0,3.0,3.0,3.0,3.0
89882,72.0,31.0,33.0,68.0,52.0,32.0,41.0,29.0,30.0,47.0,...,35.0,39.0,68.0,69.0,68.0,15.0,10.0,10.0,12.0,15.0


In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [49]:
lin_model1 = LinearRegression()
lin_model1.fit(X_train, y_train)

LinearRegression()

In [50]:
y_pred1 = lin_model1.predict(X_test)
y_pred1

array([[79.01520896],
       [78.22603517],
       [74.33510149],
       ...,
       [69.87301914],
       [68.27069969],
       [66.37610715]])

In [52]:
y_pred1.mean()

68.597019183854

In [53]:
df['overall_rating'].mean()

68.6000152886831

In [80]:
# Root Mean Squared Error
rmse_lin = math.sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred1))
print(rmse_lin)

2.7886874646658746


In [77]:
rf1 = RandomForestRegressor()
rf1.fit(X_train, y_train.values.flatten())

RandomForestRegressor()

In [78]:
y_pred2 = rf1.predict(X_test)
y_pred2

array([78.6       , 77.81      , 74.78      , ..., 72.83416667,
       69.305     , 63.906     ])

In [79]:
y_pred2.mean()

68.60118758957422

In [81]:
rmse_rf = math.sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred2))
print(rmse_rf)

1.3415178752503538
